In [80]:
import requests
from bs4 import BeautifulSoup
import datetime
import traceback
from tabulate import tabulate
from email import encoders
from email.header import Header
from email.mime.text import MIMEText
from email.utils import parseaddr, formataddr
import smtplib
import json

In [81]:
resp = requests.get('http://vip.stock.finance.sina.com.cn/corp/go.php/vRPD_NewStockIssue/page/1.phtml')
resp.encoding = 'gb18030' 
soup = BeautifulSoup(resp.text, 'lxml')

In [82]:
stocks = [['证券代码', '申购代码', '证券简称', '上网发行日期', '上市日期', 
          '发行数量(万股)', '上网发行数量(万股)', '发行价格(元)', '市盈率']]

In [83]:
for line in soup.find(id='NewStockTable').find_all('tr')[3:]:
    try:
        info = [td.get_text().strip() for td in line.find_all('td')]
        ipo_date = info[3]
        tomorrow = (datetime.date.today() + datetime.timedelta(days=1)).isoformat()
        if ipo_date == tomorrow:
            stocks.append(info[:-6])
        elif ipo_date < tomorrow:
            break
    except:
        traceback.print_exc()
        continue

In [85]:
with open('smtp.json') as f:
    config = json.load(f)

msg = MIMEText(tabulate(stocks), 'plain', 'utf-8')
msg['From'] = config['from']
msg['To'] = ','.join(config['to'])
msg['Subject'] = Header(u'明日新股', 'utf-8').encode()

server = smtplib.SMTP_SSL(config['server'], 465)
server.set_debuglevel(1)
server.login(config['from'], config['password'])
server.sendmail(from_addr, config['to'], msg.as_string())
server.quit()

send: 'ehlo [192.168.1.2]\r\n'
reply: b'250-mail\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH LOGIN PLAIN \r\n'
reply: b'250-AUTH=LOGIN PLAIN\r\n'
reply: b'250-coremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2UrGzxT5UCa0xDrUUUUj\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250 8BITMIME\r\n'
reply: retcode (250); Msg: b'mail\nPIPELINING\nAUTH LOGIN PLAIN\nAUTH=LOGIN PLAIN\ncoremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2UrGzxT5UCa0xDrUUUUj\nSTARTTLS\n8BITMIME'
send: 'AUTH PLAIN AGR1eml0b25nODExQDEyNi5jb20ATmV0TEBUZWNoRVoyOTk=\r\n'
reply: b'235 Authentication successful\r\n'
reply: retcode (235); Msg: b'Authentication successful'
send: 'mail FROM:<duzitong811@126.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'rcpt TO:<duzitong811@126.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<

(221, b'Bye')